In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [2]:
# تنظیم device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# پیش‌پردازش تصاویر مناسب برای MobileNet
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # MobileNet نیز ورودی 224x224 می‌خواهد
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# بارگذاری داده‌ها
train_data = datasets.ImageFolder('dataset/train', transform=transform)
test_data = datasets.ImageFolder('dataset/test', transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [3]:
# بارگذاری MobileNetV2 از torchvision
model = models.mobilenet_v2(pretrained=True)

# Freeze کردن همه لایه‌ها (اختیاری)
for param in model.parameters():
    param.requires_grad = False

# جایگزینی لایه‌ی طبقه‌بندی نهایی (classifier)
model.classifier[1] = nn.Linear(model.last_channel, 2)  # 2 کلاس: سگ و گربه

model = model.to(device)

# تابع هزینه و بهینه‌ساز
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier[1].parameters(), lr=0.001)

c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\user/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth
100.0%


In [5]:
# حلقه آموزش
for epoch in range(5):
    model.train()
    total = 0
    correct = 0
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Epoch {epoch+1}, Loss: {running_loss:.4f}, Accuracy: {acc:.2f}%")

c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\PIL\TiffImagePlugin.py:935: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Epoch 1, Loss: 88.6664, Accuracy: 95.56%
Epoch 2, Loss: 70.5917, Accuracy: 96.51%
Epoch 3, Loss: 67.6916, Accuracy: 96.64%
Epoch 4, Loss: 70.9864, Accuracy: 96.70%
Epoch 5, Loss: 73.2982, Accuracy: 96.49%


In [6]:
# ارزیابی روی داده‌های تست
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 98.31%
